In [1]:
# from sklearn.impute import SimpleImputer

# def impute(X):
#     imputer = SimpleImputer(strategy='most_frequent')
#     imputer.fit(processed_train)
#     X['public_meeting'] = imputer.transform(X['public_meeting'])
#     X['permit'] = X['permit'].fillna(False)
#     X['scheme_management']
#     return X

In [ ]:
# processed_test = pd.DataFrame( impute(processed_test), columns=processed_test.columns)
# processed_train = pd.DataFrame( impute(processed_train), columns=processed_train.columns)

In [ ]:
# from sklearn.preprocessing import StandardScaler, RobustScaler
# def standardize(X):
#     # Features to standardize
#     standardize_cols = ['amount_tsh', 'gps_height', 'longitude', 
#                         'latitude', 'population', 'pump_age', 'pop*amount_tsh', 'pop/amount_tsh', 
#                         'gps_height**2', 'gps_height**3', 'latitude*height*amount', 'latitude*height',
#                        'gps_height_binned', 'amount_tsh_binned', 'longitude_binned', 'latitude_binned']
    
#     # Silence Data Conversion warning
#     X[standardize_cols] = X[standardize_cols].astype(float)
    
#     # Fit and transform scaler
#     scaler = RobustScaler()
#     scaler.fit(train[standardize_cols])
#     scaled = pd.DataFrame( scaler.transform(X[standardize_cols]) )
    
#     # Add back column names
#     for i in range(len(standardize_cols)):
#         scaled = scaled.rename(columns={i:standardize_cols[i]})
        
#     # Drop non-standardized columns
#     X = X.drop(columns=standardize_cols)
    
#     # Concat scaled features with rest of features
#     X = pd.concat([X, scaled], axis=1)
    
#     return X

In [ ]:
# train = standardize(train)
# test = standardize(test)

# train.shape, test.shape

In [ ]:
# def target_encode(X):
#     temp = X.copy()
#     # Features to target encode
#     target_cols = ['date_recorded_month', 'date_recorded_year', 
#                     'basin', 'region', 'extraction_type_class', 
#                     'management_group', 'payment', 'quality_group', 
#                     'quantity', 'source_type', 'source_class', 
#                     'waterpoint_type', 'scheme_management']
    
#     # Convert all relevant cols to category datatype (for encoder)
#     X[target_cols] = X[target_cols].astype('category')
    
#     # Initialize and transform relevant features
#     encoder = ce.TargetEncoder(smoothing=5, min_samples_leaf=5)
    
#     # Note, train hardcoded to avoid overfitting test data
#     encoder.fit(train[target_cols], target['encoded'])
#     encoded = encoder.transform(X[target_cols])
    
#     # Add Target Encoded features back to features DataFrame
#     #X = X.drop(columns=target_cols)
#     X = pd.concat([temp, encoded], axis=1)
    
#     return X## Polynomial Features

# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(interaction_only=False)
# poly.fit(processed_train)
# poly_train = poly.transform(processed_train)
# poly_test = poly.transform(processed_test)
# poly_train.shape, poly_test.shape

# poly_train = pd.DataFrame(poly_train)
# corr = pd.concat([poly_train, target['encoded']], axis=1)
# corrs = corr.corr()
# corrs['encoded'].nlargest(20)

# corrs > .5

In [ ]:
## Polynomial Features

# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(interaction_only=False)
# poly.fit(processed_train)
# poly_train = poly.transform(processed_train)
# poly_test = poly.transform(processed_test)
# poly_train.shape, poly_test.shape

# poly_train = pd.DataFrame(poly_train)
# corr = pd.concat([poly_train, target['encoded']], axis=1)
# corrs = corr.corr()
# corrs['encoded'].nlargest(20)

# corrs > .5

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=15)

# pca.fit(poly_train)

# pca_train = pca.transform(poly_train)
# pca_test = pca.transform(poly_test)

# def pca_to_df(pca_array):
#     pca_df = pd.DataFrame(pca_array)
#     for col in pca_df.columns:
#         pca_df = pca_df.rename(columns={
#             col:'PCA_Poly' + str(col) 
#         })
#     return pca_df

# pca_train = pca_to_df(pca_train)
# pca_test = pca_to_df(pca_test)

In [ ]:
# from sklearn.cluster import KMeans

# K = 15
# sum_of_squared_distances = []

# km = KMeans(n_clusters=K)
# km = km.fit(processed_train)
# sum_of_squared_distances.append(km.inertia_)

# train_clusters = km.predict(processed_train)
# test_clusters = km.predict(processed_test)

# test_clusters = pd.Series(test_clusters, name='clusters')
# train_clusters = pd.Series(train_clusters, name='clusters')

In [ ]:
## Concatenating DataFrame

# mega_train = pd.concat([processed_train, pca_train, train_clusters], axis=1)
# mega_test = pd.concat([processed_test, pca_test, test_clusters], axis=1)

# mega_train.shape, mega_test.shape

In [ ]:
# Logistic Regression 

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

lr = LogisticRegression(solver='lbfgs')

lr_scores = cross_validate(lr, 
                        processed_train, 
                        target['status_group'], 
                        return_train_score=True,
                        return_estimator=True,
                        scoring='accuracy', 
                        n_jobs=-1,
                        cv=3)

pd.DataFrame(lr_scores)

lr_scores['estimator'][1]

lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

lr.fit(processed_train, target['status_group'])

classification_metrics(lr, processed_train, target['status_group'])

In [ ]:
# from sklearn.metrics import accuracy_score

# scores = cross_validate(grboost, 
#                         processed_train, 
#                         target['status_group'],
#                         verbose=10,
#                         return_train_score=True,
#                         return_estimator=True,
#                         scoring='accuracy', 
#                         n_jobs=-1,
#                         cv=3)

# pd.DataFrame(scores)

# from sklearn.model_selection import RandomizedSearchCV

# param_distributions = {
#     'n_estimators': [200, 300, 400],
#     'max_depth': [15, 25, 30],
# }

# search = RandomizedSearchCV(model, 
#                            param_distributions=param_distributions,
#                            n_iter=8,
#                            scoring='accuracy', 
#                            verbose=10,
#                            cv=3,
#                            n_jobs=-1,
#                            return_train_score=True)

# search.fit(final_train, target['status_group'])

# search.best_estimator_

# search.best_score_

### GradientBoostingClassifier 

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(verbose=10, n_iter_no_change=10)

param_distributions = {
    'n_estimators': [200, 300],
    'max_depth': [15, 25],
}

search = RandomizedSearchCV(gbc, 
                           param_distributions=param_distributions,
                           n_iter=8,
                           scoring='accuracy', 
                           verbose=10,
                           cv=3,
                           n_jobs=-1,
                           return_train_score=True)

search.fit(final_train, target['status_group'])

search.best_estimator_

### SVC

from sklearn.svm import SVC

svc = SVC(kernel='linear')

param_distributions = {
    'C': [0.1, 1],
    'gamma': [0.01, 'auto'],
    'kernel': ['linear', 'rbf']
}

svc_search = RandomizedSearchCV(svc, 
                           param_distributions=param_distributions,
                           n_iter=8,
                           scoring='accuracy', 
                           verbose=10,
                           cv=3,
                           n_jobs=-1,
                           return_train_score=True)

svc_search.fit(final_train, target['status_group'])

svc.best_estimator_

### K Nearest Neighbors

from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_jobs=-1)

param_distributions = {
    'n_neighbors': [3, 5, 7],
    'p': [1, 2]
}

kn_search = RandomizedSearchCV(kn, 
                           param_distributions=param_distributions,
                           n_iter=8,
                           scoring='accuracy', 
                           verbose=10,
                           cv=3,
                           n_jobs=-1,
                           return_train_score=True)

kn_search.fit(final_train, target['status_group'])

## Best Estimators

rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=25, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

gbc = GradientBoostingClassifier(n_estimators=300, max_depth=15, n_iter_no_change=10)

knn = KNeighborsClassifier(n_neighbors=7, p=1, n_jobs=-1)

svc = SVC(gamma='auto', C=1.0)

## Voting Classifier 

In [ ]:
def reverse_cardinality_check(n, df):
  """
  Given a cardinality limit (n) and a dataframe this function will search the
  dataframe for features above the cardinality limit, then create a dict
  from the results
  """
  
  feature_list = []
  
  cardinality_value = []
  
  for _ in range(len(df.columns)):
    if len(df[df.columns[_]].value_counts()) > n:
      
      feature_list.append(df.columns[_])
      
      cardinality_value.append(len(df[df.columns[_]].value_counts()))
                               
        
  feature_dict = dict(zip(feature_list, cardinality_value))
  
  return feature_dict

In [3]:
import pandas as pd

In [8]:
a = pd.DataFrame(columns=[1,2,3,4,5], index=[6,7,8], data=[[3,4,5,6,7]])


b = pd.DataFrame(columns=[1,4,3,2,5], index=[6,7,8], data=[[3,6,5,4,7]])
b

,1,4,3,2,5
6,3,6,5,4,7
7,3,6,5,4,7
8,3,6,5,4,7


In [9]:
a, b = a.align(b, join='left', axis=1)

In [10]:
a

,1,2,3,4,5
6,3,4,5,6,7
7,3,4,5,6,7
8,3,4,5,6,7


In [11]:
b

,1,2,3,4,5
6,3,4,5,6,7
7,3,4,5,6,7
8,3,4,5,6,7


In [ ]:
train_num['distance'] = np.sqrt((train_num['longitude'] - mean_long_train)**2 + (train_num['latitude'] - mean_lat_train)**2) test_num['distance'] = np.sqrt((test_num['longitude'] - mean_long_test)**2 + (test_num['latitude'] - mean_lat_test)**2)  train_num['distance_height'] = np.sqrt((train_num['gps_height']**2 + train_num['longitude'] - mean_long_train)**2 + (train_num['latitude'] - mean_lat_train)**2) test_num['distance_height'] = np.sqrt((test_num['gps_height']**2 + test_num['longitude'] - mean_long_test)**2 + (test_num['latitude'] - mean_lat_test)**2)